In [ ]:
import pydsm
from pydsm import postpro

# Dask related functions

In [ ]:
import dask
from dask.distributed import Client, LocalCluster

class DaskCluster:
    def __init__(self):
        self.client=None
    def start_local_cluster(self):
        cluster = LocalCluster(threads_per_worker=1, memory_limit='4G') # threads_per_worker=1 needed if using numba :(
        self.client = Client(cluster)
    def stop_local_cluster(self):
        self.client.shutdown()
        self.client=None

def run_all(processors):
    tasks=[dask.delayed(postpro.run_processor)(processor) for processor in processors]
    dask.compute(tasks)

In [ ]:
cluster = DaskCluster()
cluster.start_local_cluster()
cluster.client

In [ ]:
#cluster.stop_local_cluster()

# Setup for EC

In [ ]:
dssfile='e:/dsm2v82_calibration/ObservedData/updated_sep-oct2020/cdec_ec_merged_sep2020.dss'
locationfile='e:/dsm2v82_calibration/LocationInfo/calibration_ec_stations.csv'
vartype='EC'
units='mmhos/cm'
study_name='Observed'
observed=True
processors=postpro.build_processors(dssfile, locationfile, vartype, units, study_name, observed)
#run_all(processors)

In [ ]:
#for p in processors:
#    postpro.run_processor(p)

In [ ]:
#%load_ext memory_profiler
#%memit run_all(processors[0:1])

In [ ]:
run_all(processors)

# Setup for FLOW

In [ ]:
dssfile='e:/dsm2v82_calibration/ObservedData/updated_sep-oct2020/cdec_flow_merged_sep2020.dss'
locationfile='e:/dsm2v82_calibration/LocationInfo/calibration_flow_stations.csv'
vartype='FLOW'
units='cfs'
study_name='Observed'
observed=True
processors=postpro.build_processors(dssfile, locationfile, vartype, units, study_name, observed)
run_all(processors)
#for p in processors:
#    postpro.run_processor(p)

# Setup for STAGE

In [ ]:
dssfile='e:/dsm2v82_calibration/ObservedData/updated_sep-oct2020/cdec_stage_merged_sep2020.dss'
locationfile='e:/dsm2v82_calibration/LocationInfo/calibration_stage_stations.csv'
vartype='STAGE'
units='feet'
study_name='Observed'
observed=True
processors=postpro.build_processors(dssfile, locationfile, vartype, units, study_name, observed)
run_all(processors)

In [ ]:
cluster.stop_local_cluster()